<a href="https://colab.research.google.com/github/emrapport/w266-final-project/blob/master/CNN_w_contraction_dict.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

11/3/2019
Overview: this was the first model I trained where I had dealt with the contraction stuff (by using dictionary to expand contractions, then removing additional apostrophes from training set). We see fewer unknown words know. Results seem relatively similar to previous iterations of model - validation accuracy bounces back and forth, hovering at around where our LR baseline was, maybe a little better. Here we see validation accuracy really dropping after a certain place - could also have to do with the fact that we're using a somewhat smaller training set. 

In [0]:
# this notebook is based off of this blog post: 
# https://realpython.com/python-keras-text-classification/#reader-comments

In [0]:
DATASET_NAME = "contractions"
MODEL_NAME = "contractions"
MAX_SEQ_LENGTH = 20
TRAINING_SET_SIZE = 2000000
VAL_SET_SIZE = 1000000
NUM_EPOCHS = 15
BATCH_SIZE = 1000

# I don't think we have a good reason to do this right now
# but we might eventually 
SAVE_TOKENIZED_DATA = False

In [2]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import HashingVectorizer
from scipy.sparse import hstack, vstack
from keras.preprocessing.text import Tokenizer
from tensorflow.keras import Sequential, layers
from keras.utils import plot_model
import pandas as pd
import numpy as np
import copy
import time
import pickle
!pip install gcsfs

pd.set_option('max_colwidth', 100)

Using TensorFlow backend.


     |████████████████████████████████| 51kB 1.9MB/s 
  Created wheel for gcsfs: filename=gcsfs-0.3.1-py2.py3-none-any.whl size=17936 sha256=c647ae19577d876a120b22f179859518f3bace5c9f41677bc3126c1f69653aac
  Stored in directory: /root/.cache/pip/wheels/9d/2b/6f/86954f0d8caa1173841e62bb780dc0f8693bd268e04a267682
Successfully built gcsfs


In [3]:
# this cell is only necessary if running in colab
project_id = 'w266-251323'
import uuid
bucket_name = 'fb-congressional-data/'
from google.colab import auth
auth.authenticate_user()
!gcloud config set project {project_id}

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Updated property [core/project].


In [4]:
train_df = pd.read_csv("gs://fb-congressional-data/contraction_expanded_data/train.csv", index_col=0)

/usr/local/lib/python3.6/dist-packages/numpy/lib/arraysetops.py:568: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [5]:
dev_df = pd.read_csv("gs://fb-congressional-data/contraction_expanded_data/dev.csv", index_col=0)

/usr/local/lib/python3.6/dist-packages/numpy/lib/arraysetops.py:568: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [0]:
# shuffle the data
# be sure to do this before you extract X's and y's!!
train_df = train_df.sample(frac=1)
dev_df = dev_df.sample(frac=1)

In [0]:
y_train = train_df.op_gender.values
y_dev = dev_df.op_gender.values

In [0]:
def turn_to_ints(li):
    final_list = []
    for gender in li:
        if gender=='M':
            final_list.append(1)
        else:
            final_list.append(0)
    return final_list
            
y_train = turn_to_ints(y_train)
y_dev = turn_to_ints(y_dev)

In [0]:
y_train = np.asarray(y_train)
y_dev = np.asarray(y_dev)

In [0]:
def get_text_list(init_list):
    sentences = []
    for sentence in init_list:
        if type(sentence) != str:
            sentences.append("")
        else:
            sentences.append(sentence)
    return sentences

new_sentences_train = get_text_list(train_df.response_text.values)
new_sentences_test = get_text_list(dev_df.response_text.values)

In [11]:
time_start = time.time()

# this is the default list of filters + apostrophe
# added because we have dealt with common contractions, so other apostrophes should mostly be possessive 
tokenizer = Tokenizer(num_words=200000, filters='!"\'#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n')
tokenizer.fit_on_texts(new_sentences_train)

X_train = tokenizer.texts_to_sequences(new_sentences_train)
X_test = tokenizer.texts_to_sequences(new_sentences_test)

vocab_size = len(tokenizer.word_index) + 1

currentTime = time.gmtime(time.time() - time_start)

#Convert the gmtime struct to a string
timeStr = time.strftime("%M minutes, %S seconds", currentTime)

print("Tokenized in {}".format(timeStr))

Tokenized in 11 minutes, 12 seconds


In [0]:
from keras.preprocessing.sequence import pad_sequences

maxlen = 20

X_train = pad_sequences(X_train, padding='post', maxlen=maxlen)
X_test = pad_sequences(X_test, padding='post', maxlen=maxlen)

In [0]:
if SAVE_TOKENIZED_DATA:
  x_train_path = 'X_train_{}.pkl'.format(DATASET_NAME)
  x_dev_path = 'X_dev_{}.pkl'.format(DATASET_NAME)
  y_train_path = 'y_train_{}.pkl'.format(DATASET_NAME)
  y_dev_path = 'y_dev_{}.pkl'.format(DATASET_NAME)

  with open(x_train_path, 'wb') as file:
      pickle.dump(X_train, file)   
  with open(x_dev_path, 'wb') as file:
      pickle.dump(X_test, file)
  with open(y_train_path, 'wb') as file:
      pickle.dump(y_train, file)
  with open(y_dev_path, 'wb') as file:
      pickle.dump(y_dev, file)

  # copy to bucket
  !gsutil cp /content/{x_train_path} gs://fb-congressional-data/test
  !gsutil cp /content/{x_dev_path} gs://fb-congressional-data/
  !gsutil cp /content/{y_train_path} gs://fb-congressional-data/
  !gsutil cp /content/{y_dev_path} gs://fb-congressional-data/

In [13]:
!gsutil cp gs://fb-congressional-data/glove* /tmp/

Copying gs://fb-congressional-data/glove.6B.100d.txt...
Copying gs://fb-congressional-data/glove.6B.200d.txt...
Copying gs://fb-congressional-data/glove.6B.300d.txt...
Copying gs://fb-congressional-data/glove.6B.50d.txt...
/ [4 files][  2.1 GiB/  2.1 GiB]  132.3 MiB/s                                   
==> NOTE: You are performing a sequence of gsutil operations that may
run significantly faster if you instead use gsutil -m cp ... Please
see the -m section under "gsutil help options" for further information
about when gsutil -m can be advantageous.

Copying gs://fb-congressional-data/glove.6B.zip...
| [5 files][  2.9 GiB/  2.9 GiB]   28.0 MiB/s                                   
Operation completed over 5 objects/2.9 GiB.                                      


In [0]:
def create_embedding_matrix(filepath, word_index, embedding_dim):
    vocab_size = len(word_index) + 1  # Adding again 1 because of reserved 0 index
    embedding_matrix = np.zeros((vocab_size, embedding_dim))

    with open(filepath) as f:
        for line in f:
            word, *vector = line.split()
            if word in word_index:
                idx = word_index[word] 
                embedding_matrix[idx] = np.array(
                    vector, dtype=np.float32)[:embedding_dim]

    return embedding_matrix

embedding_dim = 100
embedding_matrix = create_embedding_matrix(
                     '/tmp/glove.6B.{}d.txt'.format(embedding_dim),
                      tokenizer.word_index, embedding_dim)

In [15]:
# trying to figure out which words are empty here
counter = 0
empty_indexes = []
for index, row in enumerate(embedding_matrix):
  if sum(row) == 0:
    empty_indexes.append(index)
    counter += 1
  if counter > 1000:
    break

for idx in empty_indexes:
  try:
    print(tokenizer.index_word[idx])
  except:
    print("No entry for {}".format(idx))


No entry for 0
stillsanders
youtu
fbid
palestinei
•
standwithrand
🇺🇸
obummer
rinos
🇺🇸🇺🇸
doyourjob
huffingtonpost
👍
washingtonpost
feelthebern
😍
lmao
killary
❤️
fbcdn
nobillnobreak
hphotos
bridenstine
randrally
hahaha
isil
randpaul
cispa
bengazi
😊
sheeple
bernieorbust
😂
neverhillary
repub
aspx
😡
thehill
1073741828
dailycaller
trump2016
libtards
obozo
thinkprogress
§
scontent
͡°
❤
politicususa
berniesanders
agsaf
westernjournalism
╬═╬
sphotos
drumpf
2366
👎
washingtontimes
worstresponders
teabaggers
hahahaha
fref
akamaihd
wenstrup
hilliary
demandavote
nobama
libtard
oboma
banksters
teaparty
koolaid
ovomit
nationalreview
infowars
notif
😍😍😍
shouldnt
🇺🇸🇺🇸🇺🇸
bernie2016
opensecrets
protectsacredlands
occupydemocrats
0bama
😍😍
daca
2fwww
devoss
nodapl
imho
♥
washingtonexaminer
dumbass
wtg
dapl
███
govtrack
obuma
cromnibus
killery
cnsnews
على
conservativetribune
😉
gofundme
lieing
😀
amodei
bucshon
🙏
benifits
democraps
bonamici
democrates
♡
p71bjjcp8
randpaul2016
felicidades
abcnews
tahmooressi
264

In [16]:
# hmmmm....
nonzero_elements = np.count_nonzero(np.count_nonzero(embedding_matrix, axis=1))
nonzero_elements / vocab_size

0.16257338124855325

In [17]:
embedding_matrix.shape

(1054084, 100)

In [18]:
model = Sequential()
model.add(layers.Embedding(vocab_size, embedding_dim, 
                           weights=[embedding_matrix], 
                           input_length=maxlen, 
                           trainable=False))
model.add(layers.Conv1D(128, 5, activation='relu'))
model.add(layers.GlobalMaxPooling1D())
model.add(layers.Dense(10, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
model.summary()

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 20, 100)           105408400 
_________________________________________________________________
conv1d (Conv1D)              (None, 16, 128)           64128     
_________________________________________________________________
global_max_pooling1d (Global (None, 128)               0         
_________________________________________________________________
dense (Dense)                (None, 10)                1290      
_________________________________________________________________
dense_1 (Dense)            

In [0]:
smaller_X_train = X_train[:TRAINING_SET_SIZE]

In [0]:
smaller_y_train = y_train[:TRAINING_SET_SIZE]

In [0]:
smaller_X_dev = X_test[:VAL_SET_SIZE]
smaller_y_dev = y_dev[:VAL_SET_SIZE]

In [22]:
try:
  time_start = time.time()

  history = model.fit(smaller_X_train, smaller_y_train,
                      epochs=NUM_EPOCHS,
                      verbose=True,
                      validation_data=(smaller_X_dev, smaller_y_dev),
                      batch_size=BATCH_SIZE)

  currentTime = time.gmtime(time.time() - time_start)

  #Convert the gmtime struct to a string
  timeStr = time.strftime("%M minutes, %S seconds", currentTime)

  print("Trained in {}".format(timeStr))

except:
  currentTime = time.gmtime(time.time() - time_start)

  #Convert the gmtime struct to a string
  timeStr = time.strftime("%M minutes, %S seconds", currentTime)

  print("Trained in {}".format(timeStr))  

Train on 2000000 samples, validate on 1000000 samples
Epoch 1/15
2000000/2000000 [==============================] - 62s 31us/sample - loss: 0.5295 - acc: 0.7489 - val_loss: 0.4560 - val_acc: 0.8348
Epoch 2/15
2000000/2000000 [==============================] - 62s 31us/sample - loss: 0.5078 - acc: 0.7590 - val_loss: 0.4567 - val_acc: 0.8276
Epoch 3/15
2000000/2000000 [==============================] - 61s 31us/sample - loss: 0.5011 - acc: 0.7621 - val_loss: 0.4470 - val_acc: 0.8392
Epoch 4/15
2000000/2000000 [==============================] - 62s 31us/sample - loss: 0.4966 - acc: 0.7642 - val_loss: 0.4701 - val_acc: 0.8248
Epoch 5/15
2000000/2000000 [==============================] - 63s 31us/sample - loss: 0.4931 - acc: 0.7656 - val_loss: 0.4494 - val_acc: 0.8324
Epoch 6/15
2000000/2000000 [==============================] - 63s 31us/sample - loss: 0.4904 - acc: 0.7669 - val_loss: 0.4662 - val_acc: 0.8199
Epoch 7/15
2000000/2000000 [==============================] - 64s 32us/sample - lo

In [23]:
loss, accuracy = model.evaluate(smaller_X_train, smaller_y_train, verbose=False)
print("Training Accuracy: {:.4f}".format(accuracy))
loss, accuracy = model.evaluate(X_test, y_dev, verbose=False)
print("Testing Accuracy:  {:.4f}".format(accuracy))


Training Accuracy: 0.7762
Testing Accuracy:  0.8051


In [0]:
# can't get this to work
plot_model(history)

In [0]:
preds = model.predict(X_test)

In [0]:
dev_df['probs'] = preds

In [0]:
def pred_to_label(row):
  if row['probs'] >= .5:
    return 'M'
  else:
    return 'W'

dev_df['preds'] = dev_df.apply(pred_to_label, axis=1)

In [0]:
wrong_preds = dev_df[dev_df.op_gender!=dev_df.preds]

In [28]:
wrong_preds.op_gender.value_counts()

W    287587
M    159320
Name: op_gender, dtype: int64

In [29]:
# total proportion of preds for women
dev_df.preds.value_counts()['W'] / len(dev_df)

0.09432087738403694

In [30]:
#random sample of mistakes
wrong_preds.head(20)

,original_idx,op_id,op_gender,post_id,responder_id,response_text,op_name,op_category,probs,preds
375657,3125748,23984618,W,82878,Mary,return to the middle ages!,Ann Wagner,Congress_Republican,0.750611,M
1983757,11033149,86569077,W,389881,Patrick,is the cost fixed or will this be another failure?,Kirsten Gillibrand,Congress_Democratic,0.747923,M
1953447,11002839,86569077,W,389564,Nell,yes.,Kirsten Gillibrand,Congress_Democratic,0.802623,M
427718,3360953,76269753,W,103742,Paul,reduce gun violence in america by giving stiffer penalties for gun violence. if someone's convi...,Dianne Feinstein,Congress_Democratic,0.756381,M
2113168,11254148,77234050,W,399620,Donald,https://scontent-a.xx.fbcdn.net/hphotos-ash3/t1.0-9/10155907_639352259453115_3858181598147954142...,Cathy McMorris Rodgers,Congress_Republican,0.530259,M
673708,4922762,46506154,M,160229,Darin,another puppet in the white house? i do not think so...go trump!,Marco Rubio,Congress_Republican,0.376635,W
426796,3360031,76269753,W,103741,Jordan,https://www.facebook.com/photo.php?fbid=629317760420431&set=a.303916709627206.79996.209960642356...,Dianne Feinstein,Congress_Democratic,0.696137,M
2049396,11098788,86569077,W,390665,Hanford,... 2-words: egg-beaters!!!,Kirsten Gillibrand,Congress_Democratic,0.630695,M
89694,2207609,38794897,M,38342,Chris,i miss your logic here? it has pertinant to the post right? you and your two houses......mass a...,Sherrod Brown,Congress_Democratic,0.479526,W
347868,2612403,59227815,M,63489,Richard,"paul ryan is a globalist, open borders stooge of the koch bros. he is a traitor to the american...",Paul Ryan,Congress_Republican,0.490249,W


In [31]:
# most confident wrong predictions where the answer was F but they thought M
# same as before - references to a male politician 
wrong_preds.sort_values('probs', ascending=False)[:20]

,original_idx,op_id,op_gender,post_id,responder_id,response_text,op_name,op_category,probs,preds
2001448,11050840,86569077,W,390046,Jay,/.what constitutional right does rand paul want repealed?/ rand paul's fetal personhood act vio...,Kirsten Gillibrand,Congress_Democratic,0.999856,M
1951491,11000883,86569077,W,389558,Sergio,thank you for doing so over mr. schumer. you are a real patriot!,Kirsten Gillibrand,Congress_Democratic,0.999834,M
2002569,11051961,86569077,W,390054,Lacy,"they are trying to keep from having to cast a vote one way or the other. of course, randy rand ...",Kirsten Gillibrand,Congress_Democratic,0.999826,M
2001084,11050476,86569077,W,390043,Margaret,"the shooter took his mother's gun. if these guns were still banned, she would not have it. mr ra...",Kirsten Gillibrand,Congress_Democratic,0.999755,M
2030227,11079619,86569077,W,390346,John,larry north do you mean senator chuck you schumer ?,Kirsten Gillibrand,Congress_Democratic,0.999743,M
1991901,11041293,86569077,W,389968,Ann,cruz and rand?,Kirsten Gillibrand,Congress_Democratic,0.999740,M
1986297,11035689,86569077,W,389917,Audrey,http://www.paul.senate.gov/?p=press_release&id=1011 i ask that you support senator rand paul wit...,Kirsten Gillibrand,Congress_Democratic,0.999737,M
2262215,13333552,62264117,W,511810,Jorge,god bless you sir. thank you for your service.,Grace Napolitano,Congress_Democratic,0.999727,M
2001443,11050835,86569077,W,390046,Jim,i missed that one jay....what constitutional right does rand paul want repealed?,Kirsten Gillibrand,Congress_Democratic,0.999690,M
2002426,11051818,86569077,W,390054,Susan,rand is a nazi.,Kirsten Gillibrand,Congress_Democratic,0.999639,M


In [32]:
# most confident wrong predictions where the answer was F but they thought M
# same as before - references to a male politician 
wrong_preds.sort_values('probs', ascending=True)[:20]

,original_idx,op_id,op_gender,post_id,responder_id,response_text,op_name,op_category,probs,preds
1373869,7792163,99469231,M,296202,Michael,as the terrorist attacks abroad continue....here at home (ignorance) on the behalf of state leve...,John McCain,Congress_Republican,0.000012,W
511576,4760630,46506154,M,159898,Elizabeth,"elizabeth warren gets red card yet trump & republicans have called her ""pocahontas"" ?",Marco Rubio,Congress_Republican,0.000074,W
108992,2264850,66658382,M,44203,Elizabeth,"elizabeth warren gets red card yet trump & republicans have called her ""pocahontas"" ?",Ted Cruz,Congress_Republican,0.000074,W
111383,2267241,66658382,M,44206,Elizabeth,"elizabeth warren gets red card yet trump & republicans have called her ""pocahontas"" ?",Ted Cruz,Congress_Republican,0.000074,W
111053,2266911,66658382,M,44205,Elizabeth,"elizabeth warren gets red card yet trump & republicans have called her ""pocahontas"" ?",Ted Cruz,Congress_Republican,0.000074,W
105268,2261126,66658382,M,44195,Elizabeth,"elizabeth warren gets red card yet trump & republicans have called her ""pocahontas"" ?",Ted Cruz,Congress_Republican,0.000074,W
106618,2262476,66658382,M,44198,Elizabeth,"elizabeth warren gets red card yet trump & republicans have called her ""pocahontas"" ?",Ted Cruz,Congress_Republican,0.000074,W
107179,2263037,66658382,M,44199,Elizabeth,"elizabeth warren gets red card yet trump & republicans have called her ""pocahontas"" ?",Ted Cruz,Congress_Republican,0.000074,W
105279,2261137,66658382,M,44195,Elizabeth,"elizabeth warren gets red card yet trump & republicans have called her ""pocahontas"" ?",Ted Cruz,Congress_Republican,0.000074,W
508630,4757684,46506154,M,159897,Elizabeth,"elizabeth warren gets red card yet trump & republicans have called her ""pocahontas"" ?",Marco Rubio,Congress_Republican,0.000074,W
